In [41]:
from pyspark.sql import SparkSession
import requests
import json
from pyspark.sql.functions import udf, col, explode
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, ArrayType
from pyspark.sql import Row

In [44]:
spark = SparkSession.builder.appName('sparkdf').getOrCreate()

22/11/03 15:49:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [70]:
spark.sparkContext.parallelize(jsondata)

ParallelCollectionRDD[92] at readRDDFromFile at PythonRDD.scala:274

In [106]:
spark = SparkSession.builder.appName('sparkdf').getOrCreate()
sparkContext=spark.sparkContext

def read_data(get, url):
    res = None
    response = requests.request(get,url)
#     jsondata = response.json()
    jdata = json.loads(response.text)
    return jdata
# df = spark.read.json(sparkContext.parallelize([jsondata]))
# df.show(truncate=True)

In [112]:
read_data("GET", "https://datausa.io/api/data?drilldowns=Nation&measures=Population")

{'data': [{'ID Nation': '01000US',
   'Nation': 'United States',
   'ID Year': 2020,
   'Year': '2020',
   'Population': 326569308,
   'Slug Nation': 'united-states'},
  {'ID Nation': '01000US',
   'Nation': 'United States',
   'ID Year': 2019,
   'Year': '2019',
   'Population': 324697795,
   'Slug Nation': 'united-states'},
  {'ID Nation': '01000US',
   'Nation': 'United States',
   'ID Year': 2018,
   'Year': '2018',
   'Population': 322903030,
   'Slug Nation': 'united-states'},
  {'ID Nation': '01000US',
   'Nation': 'United States',
   'ID Year': 2017,
   'Year': '2017',
   'Population': 321004407,
   'Slug Nation': 'united-states'},
  {'ID Nation': '01000US',
   'Nation': 'United States',
   'ID Year': 2016,
   'Year': '2016',
   'Population': 318558162,
   'Slug Nation': 'united-states'},
  {'ID Nation': '01000US',
   'Nation': 'United States',
   'ID Year': 2015,
   'Year': '2015',
   'Population': 316515021,
   'Slug Nation': 'united-states'},
  {'ID Nation': '01000US',
   'N

In [121]:
schema = StructType([StructField("data", ArrayType(StructType([
      StructField("ID Nation", StringType()),
      StructField("Nation", StringType())]) 
))])
    
    



In [ ]:
def get_all_columns_from_schema(source_schema):
  branches = []
  def inner_get(schema, ancestor=None):
    if ancestor is None: ancestor = []
    for field in schema.fields:
      branch_path = ancestor+[field.name]     
      if isinstance(field.dataType, StructType):    
        inner_get(field.dataType, branch_path) 
      else:
        branches.append(branch_path)
        
  inner_get(source_schema)
        
  return branches

def collapse_column(source_df, source_column):
    column_name = ""
    if isinstance(source_column, Column):
      column_name = source_column.name
    else:
      column_name = source_column

    return collapse_columns(source_df.schema, column_name)

def collapse_columns(source_schema, columnFilter=None):
  _columns_to_select = []
  if columnFilter is None: columnFilter = ""
  _all_columns = get_all_columns_from_schema(source_schema)
  for column_collection in _all_columns:
    if (len(columnFilter) > 0) & (column_collection[0] != columnFilter): 
        continue
        
        select_column_collection = ['`%s`' % list_item for list_item in column_collection]    
    
    if len(column_collection) > 1:
      _columns_to_select.append(col('.'.join(select_column_collection)).alias('_'.join(column_collection)))
    else:
      _columns_to_select.append(col(select_column_collection[0]))

  return _columns_to_select

def collapse_to_dataframe(source_df):
  return source_df.select(collapse_columns(source_df.schema))

        


In [122]:
udf_executeRestApi = udf(read_data, schema)

In [123]:
from pyspark.sql import Row
headers = {
    'content-type': "application/json"
}
body = json.dumps({
})
RestApiRequestRow = Row("verb", "url")
request_df = spark.createDataFrame([
            RestApiRequestRow("get", "https://datausa.io/api/data?drilldowns=Nation&measures=Population")
          ])

In [124]:
result_df = request_df \
             .withColumn("result", udf_executeRestApi(col("verb"), col("url")))

In [125]:
df = result_df.select(explode(col("result.data")).alias("results"))
df.select(collapse_columns(df.schema)).show()

+-----------------+--------------+
|results_ID Nation|results_Nation|
+-----------------+--------------+
|          01000US| United States|
|          01000US| United States|
|          01000US| United States|
|          01000US| United States|
|          01000US| United States|
|          01000US| United States|
|          01000US| United States|
|          01000US| United States|
+-----------------+--------------+



In [61]:
spark.stop()